In [641]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import scipy
from scipy import constants
from scipy import stats
from scipy import optimize as fit

In [707]:
#initializing all the variables

m = 1 #mass of the particle
w = 1 #frequency of oscillation
Length = 43 #length of
n = 500 #number of position sites
h_bar = 1 #setting the frequency, mass, and hbar to be 1 for convenience
a = 1 #amplitude of driving
gamma = 200 #frequency of driving, want this to be much bigger than w

In [678]:
#Numerically by Perturbation

def KP_Energies(hbar, mass, frequency, length, position_sites, amplitude, driving_frequency):
    
    #analytically calculated for ladder operator method
    w1_tilde = np.emath.sqrt((2*(frequency**2))+((4*(amplitude**2)*(driving_frequency**2))/(mass*(length**2)))) 
    w2_tilde = np.emath.sqrt(((-2)*(frequency**2))+((4*(amplitude**2)*(driving_frequency**2))/(mass*(length**2))))

    #initializing derivative and energy

    #creating x derivative operator: -1 on diagonal, +1 on +1 diagonal and +1 in bottom left corner, divide by dx
    #dx = Length / n
    #in the end want derivative operator to give us (f(x1) - f(x0))/delta t

    diag_1_x = (-1)*np.diag(np.diag(np.ones((position_sites,position_sites))))
    diag_2_x = np.diag(np.diag(np.ones((position_sites,position_sites)),k=1), k=1)

    derivative_x = diag_1_x.astype(int) + diag_2_x.astype(int)
    derivative_x[position_sites-1,0] = 1
    derivative_x = (position_sites/length)*derivative_x

    #known formulation for second derivative
    second_derivative_x = (position_sites/length)*(derivative_x + np.transpose(derivative_x)) 

    #now creating total position space which ranges from -L to L
    position_KP = np.linspace(-length,length,position_sites)

    position1_KP = np.zeros((position_sites,position_sites))
    position2_KP = np.zeros((position_sites,position_sites))

    #shifting to create two separate potentials (without overlap) at -L/2 and +L/2
    for i in range(position_sites):
        if i < position_sites//2:
            position1_KP[i,i] = position_KP[i] + length/2
        if i >= position_sites//2:
            position2_KP[i,i] = position_KP[i] - length/2

    position_squared1_KP = position1_KP@np.transpose(position1_KP)
    position_squared2_KP = position2_KP@np.transpose(position2_KP)

    #putting it all together using known formulas to calculate the Energy of the entire system
    KineticEnergy_KP = -((hbar**2)/(2*mass))*((position_sites/length)**2)*second_derivative_x
    PotentialEnergy1_KP = 0.5*mass*(w1_tilde**2)*position_squared1_KP
    PotentialEnergy2_KP = 0.5*mass*(w2_tilde**2)*position_squared2_KP

    Energy1_KP = KineticEnergy_KP + PotentialEnergy1_KP
    Energy2_KP = KineticEnergy_KP + PotentialEnergy2_KP

    Energy_KP = Energy1_KP + Energy2_KP
    return Energy_KP

In [708]:
#calculating the spectrum of numerically calculated energy data
Energy_data = KP_Energies(h_bar, m, w, Length, n, a, gamma)

Eigenvalues_KP,Eigenvectors_KP = np.linalg.eig(Energy_data)

x_data_KP = range(n)
y_data_KP = np.sort(np.real(Eigenvalues_KP))

In [703]:
#fit to linear model (based on analytically computed spectrum)

parameters1 = stats.linregress(x_data_KP, y_data_KP)

In [690]:
#plotting eigenvalues

plt.figure(figsize = (10,8))

plt.scatter(x_data_KP,y_data_KP, s = 20, marker = ".", color = 'orange', label = "Eigenvalue")

plt.plot(x_data_KP, parameters1.slope*x_data_KP + parameters1.intercept, color = 'red', linewidth = 3, label = "Linear Curve Fit")

plt.text(0.02,0.85,"$r^2$ = " + str(parameters1.rvalue**2), bbox=dict(facecolor='white', alpha = 0.2), transform=plt.gca().transAxes)
plt.text(0.02,0.8,"Curve fit: y = " + str(parameters1.slope) + " x + " + str(parameters1.intercept), bbox=dict(facecolor='white', alpha = 0.2), transform=plt.gca().transAxes)
plt.text(0.02,0.75,"L = 43" + "; n = 1000" + "; a = 1" + "; $\gamma$ = 1000", bbox=dict(facecolor='white', alpha = 0.2), transform=plt.gca().transAxes)

plt.xlabel("Eigenvalue Index")
plt.ylabel("Eigenvalue")
plt.title("Eigenvalues of Numerical Quantized Kapitza Pendulum")
plt.legend()

plt.show()

In [674]:
#performing a linear fit to the data for many different driving frequencies (to test the breakdown of linearity)
#quality of linear fit discerned by r^2

gamma_range = np.linspace(1,2000,100)
r2_array = np.zeros(len(gamma_range))

for i in gamma_range:
    Energy_data = KP_Energies(1, m, w, 43, n, 1, i)

    Eigenvalues_KP,Eigenvectors_KP = np.linalg.eig(Energy_data)

    x_data_KP = range(n)
    y_data_KP = np.sort(np.real(Eigenvalues_KP))
    
    parameters = stats.linregress(x_data_KP, y_data_KP)
    
    r2_array[np.where(gamma_range == i)] = parameters.rvalue**2

In [676]:
#plotting r^2 vs gamma

plt.figure(figsize = (10,8))

plt.scatter(gamma_range,r2_array, s = 40, marker = "o", color = 'orange', label = "$r^2$ Value")
#plt.plot(gamma_range, gaussian(gamma_range, results[0], results[1], results[2], results[3]), color = 'red', linewidth = 3, label = "Gaussian Curve Fit")

plt.text(0.02,0.95,"L = 43" + "; n = 500" + "; a = 1" + "; $\gamma$ = [0,2000]", bbox=dict(facecolor='white', alpha = 0.2), transform=plt.gca().transAxes)

plt.xlabel("$\gamma$ Value")
plt.ylabel("$r^2$ Value")
plt.title("$r^2$ Value vs $\gamma$ Value")
plt.legend()

plt.show()

Now check using analytically calculated solutions

In [682]:
def AnalyticKP_Energies(hbar, mass, frequency, length, position_sites, amplitude, driving_frequency):
    
    #analytically calculated for ladder operator method
    w1_tilde = np.emath.sqrt((2*(frequency**2))+((4*(amplitude**2)*(driving_frequency**2))/(mass*(length**2)))) 
    w2_tilde = np.emath.sqrt(((-2)*(frequency**2))+((4*(amplitude**2)*(driving_frequency**2))/(mass*(length**2))))

    #a and a dagger are known for the ladder operator method, just coding them as is then
    a_initialize = np.diag(np.diag(np.ones((position_sites,position_sites)),k=1),k=1)
    row_index = np.sqrt(np.arange(position_sites))
    a1_H = a_initialize*row_index
    a1_dagger_H = np.transpose(a1_H)

    #calculating the Hamiltonian for the two different states (centered at -L/2 and +L/2)
    H1 = (hbar*(w1_tilde)/2)*(a1_dagger_H@a1_H + 0.5*np.identity(position_sites))
    H2 = (hbar*(w2_tilde)/2)*(a1_dagger_H@a1_H + 0.5*np.identity(position_sites))

    #necessary to cancel any overlap between the Hamiltonians (we want two separate states)
    for i in range(position_sites):
        if i >= position_sites//2:
            H1[i,i] = 0
            H2[i,i] = H2[i-position_sites//2,i-position_sites//2]
    for i in range(position_sites):
        if i < position_sites//2:
            H2[i,i] = 0

    #Sum for energy of entire system
    H_final = H1 + H2
    return H_final

In [709]:
#calculating spectrum of analytically calculated energies
Energy_data2 = AnalyticKP_Energies(h_bar, m, w, Length, n, a, gamma)
Eigenvalues_H, Eigenvectors_H = np.linalg.eig(Energy_data2)

x_data_H = range(n)
y_data_H = np.sort(np.abs(Eigenvalues_H))

In [699]:
#linear regression to calculate slope, rescale by ratio of slopes to make the analytic and numerical spectra comparable
parameters2 = stats.linregress(x_data_H, y_data_H)
rescale_factor = parameters1.slope / parameters2.slope

In [693]:
plt.figure(figsize = (10,8))

#plotting rescaled analytically calculated spectrum
plt.scatter(x_data_H,y_data_H*rescale_factor, s=20,label = 'Eigenvalue', color = "orange", marker = ".")
plt.plot(x_data_H, rescale_factor*(parameters2.slope*x_data_H + parameters2.intercept), color = 'red', linewidth = 3, label = "Linear Curve Fit")

plt.text(0.02,0.85,"L = 43" + "; n = 1000" + "; a = 1" + "; $\gamma$ = 1000", bbox=dict(facecolor='white', alpha = 0.2), transform=plt.gca().transAxes)
plt.xlabel("Eigenvalue Index")
plt.ylabel("Eigenvalue")
plt.title("Eigenvalues of Analytic Quantized Kapitza Pendulum")
plt.legend()

plt.show()

Finally, plot the difference between the two spectra to show comparability

In [710]:
#calculating the error term between numerical and analytic eigenvalues

#calculating the variance (two-point function) and using it as our y-value
difference = np.abs((y_data_KP - (rescale_factor*y_data_H))/(y_data_H*rescale_factor))

mean_difference = np.mean(difference)

plt.figure(figsize = (10,8))

plt.scatter(x_data_H, (difference - mean_difference)**2/(mean_difference**2), s=10, marker = "o", label = "Variance", color = "orange")
#plt.plot(x_data_H, difference**2, label = "Difference", color = "orange")

plt.xlabel("Eigenvalue Index")
plt.ylabel("Variance")
plt.legend()
plt.title("Variance Between Numerically and Analytically Calculated Eigenvalues (Using the two-point function)")

plt.show()